# Naïve Bayes Classifier

The training set consists of 1000 rows and 55 columns. Each row corresponds to one email message. The first column is the _response_ variable and describes whether a message is spam (1) or ham (0). The remaining 54 columns represent _feature_ _vectors_ that are used to train the classifier. These features correspond to 54 different keywords (such as "money", "free", and "receive") and special characters (such as ":", "!", and "$"). A feature has the value "1" if the keyword appears in the message and "0" otherwise.

## The model
The [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier distinguishes between two classes:

* **$C = 1$ for spam messages **
* **$C = 0$ for ham messages. **


The classifier builds a model for the probability $P(C=c\ |\ message)$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate

\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \  P(C=c\ |\ message).
\end{equation}

Using Bayes' rule we can write

\begin{equation}
P(C=c\ |\ message) = \frac{P(message\ |\ C=c)P(C=c)}{P(message\ |\ C=1)P(C=1) + P(message\ |\ C=0)P(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it to get

\begin{equation}
P(C=c\ |\ message) \propto P(message\ |\ C=c)P(C=c).
\end{equation}


Messages are represented using a binary [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as $\mathbf{w} = (w_1, ..., w_k)$, where $w_i = 1$ if the word $w_i$ appears in the message and $w_i = 0$ otherwise. We assume **class-conditional independence between occurences of known words** and can therefore write 

\begin{equation}
P(message\ |\ C=c) = \prod_{i = 1}^k P(w_i\ |\ C=c).
\end{equation}

The classifier now can be written as follows :

\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ [ P(C=c)   \prod_{i = 1}^k P(w_i\ |\ C=c) ].
\end{equation}


### Multinomial Naïve Bayes

Different naïve Bayes models differ in their distributional assumptions of $P(w\ |\ C=c)$, that is, the conditional likelihood of a word $w$ given a class $c$. We will model $P(w\ |\ C=c)$ using a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution). Intuitively, the multinomial distribution assumes that the words of a message are "drawn" independently from a bag of $k$ different keywords. Depending on the class membership $c$, each keyword has a probability $\theta_{class, word}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment... $\}$, if the training data was mostly gathered from emails of researchers.

Both the class priors $P(C=c)$ and the class-conditional likelihoods $\theta_{c, w} = P(w\ |\ C=c)$ have to be estimated from the training data. The parameters $\theta_{c, w}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$, that is,

\begin{equation}
\theta_{c, w} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}

where $n_{c, w}$ is the number of times the word $w$ appears in messages of class $c$ in the training set, $n_{c}$ is the total count of words for all messages of class $c$, and $k$ is the number of features (key-words).

The likelihood of observing $\mathbf{w}$ in a message of class $c$ is proportional to
\begin{equation}
P(\mathbf{w}\ |\ C=c) \propto \prod_{i = 1}^k  (\theta_{c, i})^{w_i}.
\end{equation}

### Increasing numerical stability
The numerical result of the classifier will often lead to small numbers, which, due to floating point representation, can cause inaccuracies, and as such, the natural logorithm of the posteriror distributions should be taken instead
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ log( P(C=c)   \prod_{i = 1}^k P(w_i\ |\ C=c) ) \\
 = \underset{c \in \{0,1\}}{argmax} \ [ log( P(C=c)) + \sum_{i = 1}^k w_i \ log(\theta_{c, i}) ].
\end{equation}


The following cell imports the training data

In [4]:
import numpy as np

training_spam = np.loadtxt(open("data/training_spam.csv"), delimiter=",")

First, we need to estimate the prior probability for each class, as to get $P(C=c)$ for $c \in \{0, 1\}$ part of the classificiation equation

In [5]:
def estimate_log_class_priors(data):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column, calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
    log(P(C=0)) and log(P(C=1))

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                 the first column contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """
    #Finds the number of elements that are of class 0 and 1, and divides them by total number of samples
    num_samples = data.shape[0]
    actual_classes = data[:,0]
    
    log_class_priors = np.array([np.log(occurrences_in_column(actual_classes, 0) / num_samples), 
                                 np.log(occurrences_in_column(actual_classes, 1) / num_samples)])
    
    return log_class_priors

def occurrences_in_column(col, needle):
    """
    Given a one dimensional numpy array, count the number of occurences
    of the given value

    :param col: the data set to filter
    :param needle: the value which to count the occurences of

    :return: the number of occurences of the needle in the data set
    """
    return col[col == needle].shape[0]

Next, we need to find the empirical class-conditional likelihoods $log(P(w_i | c_j))$ for all words $w_i$ and both classes ($j \in {0, 1}$), where $\theta_{c, w} = P(w\ |\ C=c)$. Since we are using laplace smoothing, we can say
\begin{equation}
\theta_{c, w} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}
where $\alpha = 1$

In [6]:
def estimate_log_class_conditional_likelihoods(data, alpha=1.0):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column and binary features, calculate the empirical
    class-conditional likelihoods, that is,
    log(P(w_i | c_j)) for all features i and both classes (j in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, n_features]

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    # Each message will have the same number of features (columns)
    num_features= data.shape[1] - 1
    
    #Placeholder
    theta = np.zeros((2,num_features))
    
    for _class in range(2):
        #Gets a list of rows which are of the given class (0 or 1)
        class_list = data[data[:,0] == _class]
        
        #Loops through each row
        for feature_vector in class_list:
            total_words = 0
            
            #Loops through each feature (column) in the vector, excluding the message identifier, i.e. index 0
            for feature in range(1, num_features + 1):
                # Gets the total number of present words for that feature
                total_words += np.sum(class_list[:, feature])
                
                #Store the numerator of {theta_c,w} equation for each word, starting at index 0
                theta[_class][feature-1] = np.log((np.sum(class_list[:, feature]) + alpha))
                
        #Finish the calculation, storing the class conditional likelihood for each class
        theta[_class] = np.divide(theta[_class], np.log(total_words + num_features*alpha))

    return theta


Now that we have gathered all of the constituent components of the classification equation from the training, we can now put the model to use by attempmting to classify messages in a data set using \begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{argmax} \ [ log( P(C=c)) + \sum_{i = 1}^k w_i \ log(\theta_{c, i}) ].
\end{equation}


In [7]:
def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    #Placeholder
    class_predictions = np.zeros(new_data.shape[0])
    
    #Loop through each row of data (each sample)
    for sample_num in range(new_data.shape[0]):
        sample = new_data[sample_num]
        max_likelihood = 0
        
        #loop through each class prior
        for _class in range(log_class_priors.size):
            likelihood = 0
            
            #Loop through each feature of the current sample
            for feature in range(sample.size):
                #Sum the conditional probabilities for each feature, i.e. the right most part of the equation
                likelihood += sample[feature] * log_class_conditional_likelihoods[_class][feature]
                
            #Add class prior
            class_likelihood = log_class_priors[_class] + likelihood
            
            #Finds the class which maximises the likelihood and store the result
            if class_likelihood > max_likelihood:
                max_likelihood = class_likelihood
                class_predictions[sample_num] = _class
                
    return class_predictions

def accuracy(y_predictions, y_true):
    """
    Calculate the accuracy.
    
    :param y_predictions: a one-dimensional numpy array of predicted classes (0s and 1s).
    :param y_true: a one-dimensional numpy array of true classes (0s and 1s).
    
    :return acc: a float between 0 and 1 
    """
    #Comapres the predictions to actual classes
    successes = np.equal(y_predictions, y_true)
    
    #Finds number of successful predictions
    successes[successes[:] == True].size
    acc = successes[successes[:] == True].size / successes.size
    
    return acc


In [13]:
# Train the classifier with out training data
log_class_priors = estimate_log_class_priors(training_spam)
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)

testing_spam = np.loadtxt(open("data/testing_spam.csv"), delimiter=",")

class_predictions = predict(testing_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)
true_classes = testing_spam[:, 0]
testing_set_accuracy = accuracy(class_predictions, true_classes)

print("Classifier accuracy:", str(testing_set_accuracy * 100) + "%")

Classifier accuracy: 81.8%


Naïve Bayes assumes that features are mutually exclusive. In the case of spam emails, this is unlikely to be the case, i.e. certain words are likely to appear in sequence. To resolve this, we can use other classification methods such as k-nearest neighbours and logistic regession 

Logistic regression is a light weight and relatively simple yet robust algorithm, which is able to classify unseen data points through the use of the logistic function, namely  $\frac{1}{(1 + e^{-wx})}$, which gives the probabiltiy of the point belonging to class 1. For a point in n-dimensional space (n features), it can be mapped to a real number by multiplying its position vector by a series of weighting coefficients, i.e. $wx = w_1 x_1 + w_2 x_2 +...+w_n x_n$. The aim of the training phase is to calculate values for these coefficients which minimises the error rate.

In [20]:
def train(training_data, model):
    """
    Train a model on the training_data

    :param training_data: a two-dimensional numpy-array with shape = [5000, 39] 
    :param model: the sklearn model to use to classify the data 
    
    :return fitted_model: any data structure that captures your model
    """  
    #Extracts the features from training data
    feature_data = training_data[:, 1:training_data.shape[1]]
    #Gets the classes for each sample
    classes = training_data[:,0]
    
    #Fits the data into the given model
    fitted_model = model.fit(feature_data, classes)  
    
    return fitted_model

In [27]:
def test(testing_data, fitted_model):
    """
    Classify the rows of testing_data using a fitted_model. 

    :param testing_data: a two-dimensional numpy-array with shape = [n_test_samples, 38]
    :param fitted_model: the output of your train function.

    :return class_predictions: a numpy array containing the class predictions for each row
        of testing_data.
    """
    class_predictions = fitted_model.predict(testing_data)
    return class_predictions

Cross validation is used to get an estimate of the model's accuracy by training the model on $k-1$ partitions of data, and testing its accuracy on the remaining partition.

In [47]:
def cross_validation(training_data, k, training_model):
    """
    Performs cross validation on the data set 

    :param training_data: the data set that is used to train the classifier
    :param k: the number of partitions that the data set should be split in to
    :param training_model: the sklearn model to train

    :return accuracy: the mean accuracy of the model to 3 s.f.
    """
    training = np.copy(training_data)
    
    #Randomise the training data
    np.random.shuffle(training)
    
    #Splits the training data into k pieces
    partitions = np.split(training, k)
    
    accuracy = 0
    for i in range(k):
        #Removes the given subarray from the list
        data = np.delete(partitions, i, 0)
        
        #Trains the model on remaining training data (k-1 bits)
        model = train(data.reshape((data.shape[1] * (k-1)),data.shape[2]), training_model)
        predictions = test(partitions[i][:,1:], model)
        
        #Calculates the accuracy for
        successes = np.equal(predictions, partitions[i][:,0])
        successes[successes[:] == True].size
        acc = successes[successes[:] == True].size / successes.size
        accuracy += acc
        
    #Takes the mean accuracy over k samples
    return round(accuracy / k, 2)

In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

data = np.loadtxt(open("data/training_data_part_2.csv"), delimiter=",")

#Get the accuracy of knn
knn_accuracy = cross_validation(data, 10, KNeighborsClassifier())
print("KNN accuracy:", str(knn_accuracy * 100) + "%")

#Get the accuracy of logistic regression
regression_accuracy = cross_validation(data, 10, LogisticRegression(solver='liblinear'))
print("Logistic regression accuracy:", str(regression_accuracy * 100) + "%")

KNN accuracy: 94.0%
Logistic regression accuracy: 93.0%
